# Hybrid Financial Intelligence System

__Use case:__ Predict whether high-volatility stocks (SMCI, CRSP, PLTR) will gain more than 3.5% over the next 5 trading days.

__Dataset:__ Daily OHLCV price data from Yahoo Finance, plus a macro factor (10-Year Treasury Yield from FRED). The target is a binary label: 1 if the 5-day forward return exceeds 3.5%, 0 otherwise.

## Importing the Dataset

In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
import os

In [ ]:
data_dir = os.path.join('..', 'data')
os.listdir(data_dir)

In [ ]:
prices_path = os.path.join(data_dir, 'prices_daily.csv')
raw_df = pd.read_csv(prices_path, parse_dates=['date'])
raw_df

In [ ]:
raw_df.shape

In [ ]:
raw_df.info()

We have daily OHLCV data for 3 tickers. Let's check what tickers are in the dataset and the date range.

In [ ]:
print('Tickers:', raw_df['ticker'].unique())
print('Date range:', raw_df['date'].min(), 'to', raw_df['date'].max())
print('Rows per ticker:')
raw_df['ticker'].value_counts()

In [ ]:
raw_df.describe()

Let's also check for missing values.

In [ ]:
raw_df.isna().sum()

No missing values in the price data. Now let's load the macro factor (10-Year Treasury Yield).

In [ ]:
macro_path = os.path.join(data_dir, 'macro_10y_yield.csv')
macro_df = pd.read_csv(macro_path, parse_dates=['date'])
macro_df

In [ ]:
macro_df.info()

We need to merge the macro data with the price data. Since bond yields are only reported on business days, we'll forward-fill to cover weekends and holidays.

In [ ]:
# Expand macro to full calendar and forward-fill gaps
full_range = pd.date_range(macro_df['date'].min(), raw_df['date'].max(), freq='D')
macro_full = macro_df.set_index('date').reindex(full_range).ffill().rename_axis('date').reset_index()

# Merge with prices
merged = raw_df.merge(macro_full, on='date', how='left')
merged

In [ ]:
merged.isna().sum()

Good, the merge went smoothly. Let's check a few rows to make sure the yield values look reasonable.

In [ ]:
merged[['date', 'ticker', 'adj_close', 'ten_year_yield']].sample(5, random_state=42)

# Exploratory Data Analysis and Visualization

In [ ]:
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

## Normalized Price Paths

To compare tickers with very different price levels, we normalize each to start at 100.

In [ ]:
price_pivot = merged.pivot_table(index='date', columns='ticker', values='adj_close')
normalized = price_pivot / price_pivot.iloc[0] * 100

fig = px.line(normalized, title='Normalized Price Paths (base = 100)')
fig.update_layout(yaxis_title='Indexed Price', xaxis_title='Date')
fig.show()

- SMCI shows the most dramatic moves, with huge rallies and sharp drawdowns
- PLTR has a more stable uptrend since late 2022
- CRSP tends to move independently from the other two
- All three are clearly high-volatility names $\rightarrow$ a higher success threshold (3.5%) makes sense here

## Histogram: Daily Returns by Ticker

In [ ]:
# Compute daily returns
merged = merged.sort_values(['ticker', 'date'])
merged['daily_return'] = merged.groupby('ticker')['adj_close'].pct_change()

fig = px.histogram(merged.dropna(subset=['daily_return']), x='daily_return', color='ticker',
                   nbins=100, opacity=0.6, barmode='overlay',
                   title='Distribution of Daily Returns by Ticker')
fig.update_layout(xaxis_title='Daily Return', yaxis_title='Count')
fig.show()

- SMCI has noticeably fatter tails $\rightarrow$ more extreme daily moves (both up and down)
- All three tickers show roughly symmetric distributions centered near 0
- The spread confirms these are genuinely high-volatility stocks

## Scatter Plot: Volume vs Absolute Daily Return

In [ ]:
merged['abs_return'] = merged['daily_return'].abs()

# Sample to keep the plot readable
sample = merged.dropna(subset=['abs_return']).sample(2000, random_state=42)

fig = px.scatter(sample, x='volume', y='abs_return', color='ticker',
                opacity=0.4, title='Volume vs Absolute Daily Return')
fig.update_layout(xaxis_title='Volume', yaxis_title='|Daily Return|')
fig.show()

- Higher volume days tend to have larger absolute returns, which makes sense
- SMCI shows some extreme outliers on both axes
- This relationship suggests volume-based features could be useful for the model

## Scatter Plot: MinTemp vs MaxTemp Equivalent (Low vs High Price)

In [ ]:
sample2 = merged.sample(2000, random_state=42)
fig = px.scatter(sample2, x='low', y='high', color='ticker',
                opacity=0.4, title='Daily Low vs High Price by Ticker')
fig.update_layout(xaxis_title='Low', yaxis_title='High')
fig.show()

- Strong linear correlation between daily low and high (as expected)
- SMCI occupies a much wider range, reflecting bigger intraday swings
- The spread between low and high within a day is essentially what ATR captures

## Treasury Yield Over Time

In [ ]:
fig = px.line(macro_full, x='date', y='ten_year_yield',
             title='10-Year Treasury Yield Over Time')
fig.update_layout(yaxis_title='Yield (%)', xaxis_title='Date')
fig.show()

- Sharp rise from 2022 onward, which coincides with the Fed rate-hiking cycle
- The yield environment shifted significantly during our data period $\rightarrow$ including it as a macro feature is justified

## Summary Statistics per Ticker

In [ ]:
summary = merged.groupby('ticker').agg(
    obs=('adj_close', 'count'),
    close_mean=('adj_close', 'mean'),
    close_std=('adj_close', 'std'),
    close_min=('adj_close', 'min'),
    close_max=('adj_close', 'max'),
    volume_mean=('volume', 'mean'),
    daily_vol=('daily_return', 'std'),
    mean_return=('daily_return', 'mean')
).round(4)
summary

- SMCI has the highest daily volatility and the widest price range
- Daily returns average close to zero for all tickers, but the standard deviations differ a lot
- These numbers confirm the high-volatility regime: typical 5-day moves can easily exceed 3.5%

# Feature Engineering

We'll compute standard technical indicators for each ticker, then build rolling-window features.

In [ ]:
import ta

## Adding Technical Indicators

We compute per ticker: RSI(14), MACD(12,26,9), Bollinger Bands(20), moving averages (50, 200), volume z-score, and ATR(14).

In [ ]:
featured_parts = []

for ticker, g in merged.groupby('ticker', group_keys=False):
    g = g.copy().sort_values('date')
    close = g['adj_close'].astype(float)
    high = g['high'].astype(float)
    low = g['low'].astype(float)
    volume = g['volume'].astype(float)

    # RSI
    g['rsi_14'] = ta.momentum.RSIIndicator(close).rsi()

    # MACD
    macd = ta.trend.MACD(close)
    g['macd'] = macd.macd()
    g['macd_signal'] = macd.macd_signal()
    g['macd_hist'] = macd.macd_diff()

    # Bollinger Bands
    bb = ta.volatility.BollingerBands(close)
    g['bb_high'] = bb.bollinger_hband()
    g['bb_low'] = bb.bollinger_lband()
    g['bb_width'] = (bb.bollinger_hband() - bb.bollinger_lband()) / close

    # Moving averages
    g['ma_50'] = close.rolling(50).mean()
    g['ma_200'] = close.rolling(200).mean()

    # Volume z-score
    g['volume_z'] = (volume - volume.rolling(50).mean()) / (volume.rolling(50).std() + 1e-9)

    # ATR (Average True Range)
    atr_ind = ta.volatility.AverageTrueRange(high=high, low=low, close=close, window=14)
    g['atr_14'] = atr_ind.average_true_range()
    g['atr_pct'] = (g['atr_14'] / close) * 100

    featured_parts.append(g)

featured = pd.concat(featured_parts, ignore_index=True)
print(f'Shape after adding indicators: {featured.shape}')
featured.head()

Let's check how many NaN values we introduced (indicators need a warm-up period).

In [ ]:
featured[['rsi_14', 'macd', 'bb_width', 'ma_50', 'ma_200', 'atr_14']].isna().sum()

MA(200) has the most missing values because it needs 200 days of data to start computing. These NaN rows will be handled when we build the rolling-window dataset.

## ATR Distribution by Ticker

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
sns.boxplot(data=featured.dropna(subset=['atr_14']), x='ticker', y='atr_14', ax=ax)
ax.set_title('ATR(14) Distribution by Ticker')
ax.set_ylabel('ATR (14-day)')
plt.tight_layout()
plt.show()

- SMCI has a much higher and more spread-out ATR, confirming its extreme volatility
- ATR-based features will help the model distinguish between normal and abnormal price action
- We also use ATR as a percentage of price (`atr_pct`) so it's comparable across tickers

## Creating the Target Variable

For each row, we compute the 5-day forward return and label it 1 if it exceeds 3.5%.

In [ ]:
HORIZON = 5
THRESHOLD = 0.035  # 3.5%

target_parts = []
for ticker, g in featured.groupby('ticker', group_keys=False):
    g = g.copy().sort_values('date')
    close = g['adj_close'].astype(float)
    future_price = close.shift(-HORIZON)
    g['future_return'] = future_price / close - 1.0
    g['target'] = (g['future_return'] > THRESHOLD).astype(int)
    target_parts.append(g)

featured = pd.concat(target_parts, ignore_index=True)
featured[['date', 'ticker', 'adj_close', 'future_return', 'target']].tail(10)

Let's see the class balance.

In [ ]:
target_counts = featured['target'].value_counts()
print(target_counts)
print(f'\nPositive class ratio: {target_counts.get(1, 0) / target_counts.sum():.4f}')

## Histogram: 5-Day Forward Returns

In [ ]:
fig = px.histogram(featured.dropna(subset=['future_return']), x='future_return',
                   nbins=80, title='Distribution of 5-Day Forward Returns')
fig.add_vline(x=THRESHOLD, line_dash='dash', line_color='red',
              annotation_text=f'Threshold = {THRESHOLD:.1%}')
fig.update_layout(xaxis_title='5-Day Forward Return', yaxis_title='Count')
fig.show()

- The distribution has fat tails on both sides, typical of high-volatility stocks
- A decent portion of observations exceed the 3.5% threshold
- There are also extreme negative returns $\rightarrow$ a long/cash strategy (no shorting) is safer

## Building the Rolling Window Dataset

Instead of using raw indicator values (which change scale over time), we compute rolling statistics over a 14-day window: mean, standard deviation, and last value for each indicator.

In [ ]:
WINDOW = 14

feature_cols = ['adj_close', 'rsi_14', 'macd', 'macd_signal', 'macd_hist',
                'bb_width', 'ma_50', 'ma_200', 'volume_z',
                'atr_14', 'atr_pct', 'ten_year_yield']

rows = []
for ticker, g in featured.groupby('ticker'):
    g = g.sort_values('date').reset_index(drop=True)
    for idx in range(WINDOW, len(g) - HORIZON):
        window = g.iloc[idx - WINDOW : idx]
        current = g.iloc[idx]

        if pd.isna(current['target']) or pd.isna(current['future_return']):
            continue

        stats = {}
        for col in feature_cols:
            if col not in window.columns:
                continue
            s = window[col].astype(float)
            stats[f'{col}_mean'] = s.mean(skipna=True)
            stats[f'{col}_std'] = s.std(skipna=True)
            last_valid = s.dropna()
            stats[f'{col}_last'] = last_valid.iloc[-1] if len(last_valid) > 0 else np.nan

        row = {
            'ticker': current['ticker'],
            'date': current['date'],
            'target': int(current['target']),
            'future_return': current['future_return'],
            **stats
        }
        rows.append(row)

dataset = pd.DataFrame(rows)

# Fill remaining NaN in features with 0
feat_cols_to_fill = [c for c in dataset.columns if c not in ['ticker', 'date', 'target', 'future_return']]
dataset[feat_cols_to_fill] = dataset[feat_cols_to_fill].fillna(0)
dataset = dataset.reset_index(drop=True)

print(f'Dataset shape: {dataset.shape}')
print(f'Features: {len(feat_cols_to_fill)}')
dataset.head()

In [ ]:
print(f'Target distribution:')
print(dataset['target'].value_counts())
print(f'\nPositive class ratio: {dataset["target"].mean():.4f}')

## Target Class Balance

In [ ]:
fig, ax = plt.subplots(figsize=(6, 4))
sns.countplot(data=dataset, x='target', ax=ax)
ax.set_title('Target Class Balance')
ax.set_xlabel('Target (1 = 5-day return > 3.5%)')
plt.tight_layout()
plt.show()

- The classes are not perfectly balanced but it's not extreme either
- We'll use `scale_pos_weight` in XGBoost to compensate for the imbalance

## Feature Correlations with Target

In [ ]:
corr_with_target = dataset[feat_cols_to_fill + ['target']].corr()['target'].drop('target')
top_corr = corr_with_target.abs().sort_values(ascending=False).head(18)
top_features = corr_with_target[top_corr.index].sort_values()

fig, ax = plt.subplots(figsize=(10, 6))
colors = ['green' if v > 0 else 'red' for v in top_features.values]
top_features.plot(kind='barh', color=colors, ax=ax)
ax.set_title('Top 18 Feature Correlations with Target')
ax.set_xlabel('Correlation')
plt.tight_layout()
plt.show()

- Some ATR and volatility-related features show decent correlation with the target
- RSI features and momentum indicators also appear useful
- No single feature is extremely predictive on its own $\rightarrow$ we need a model that can combine them

# Dividing Data into Training and Test Sets

Since this is time-series data, we split chronologically: everything before 2023 is training, everything from 2023 onward is the test set. This prevents data leakage.

In [ ]:
CUTOFF = '2023-01-01'

dataset = dataset.sort_values('date').reset_index(drop=True)

train_df = dataset[dataset['date'] < CUTOFF].copy()
test_df = dataset[dataset['date'] >= CUTOFF].copy()

print(f'Training set: {len(train_df)} rows')
print(f'Test set:     {len(test_df)} rows')

In [ ]:
# Let's see the date ranges
print(f'Train: {train_df["date"].min().date()} to {train_df["date"].max().date()}')
print(f'Test:  {test_df["date"].min().date()} to {test_df["date"].max().date()}')

## Identifying Input and Target Columns

In [ ]:
# We exclude metadata columns from features
exclude = ['ticker', 'date', 'target', 'future_return']
input_cols = [c for c in dataset.columns if c not in exclude]
target_col = 'target'

print(f'Number of input features: {len(input_cols)}')
print(f'Target column: {target_col}')

In [ ]:
train_inputs = train_df[input_cols].copy()
train_targets = train_df[target_col].copy()

test_inputs = test_df[input_cols].copy()
test_targets = test_df[target_col].copy()

In [ ]:
train_inputs.describe()

The features have very different scales (some are in hundreds, some near zero). We need to scale them.

## Scaling Numeric Features

We use MinMaxScaler to bring everything into the [0, 1] range. We fit the scaler on the training set only to avoid leaking test information.

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()
scaler.fit(train_inputs)

train_inputs_scaled = pd.DataFrame(scaler.transform(train_inputs), columns=input_cols, index=train_inputs.index)
test_inputs_scaled = pd.DataFrame(scaler.transform(test_inputs), columns=input_cols, index=test_inputs.index)

In [ ]:
# Verify the scaling worked
train_inputs_scaled.describe()

The minimum is now 0 (or close) and the maximum is 1 for all columns in the training set. The test set might slightly exceed [0, 1] if test values go outside the training range, which is expected.

# Training an XGBoost Model

In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report, precision_score

We train with 5-fold time-series cross-validation. This respects the temporal order: each fold uses only past data for training.

In [ ]:
# Handle class imbalance
n_pos = train_targets.sum()
n_neg = len(train_targets) - n_pos
spw = n_neg / n_pos
print(f'Negative samples: {n_neg}, Positive samples: {n_pos}')
print(f'scale_pos_weight: {spw:.2f}')

In [ ]:
SEED = 42

tscv = TimeSeriesSplit(n_splits=5)
best_precision = -1
best_model = None

X_train = train_inputs_scaled.values
y_train = train_targets.values

for fold, (tr_idx, val_idx) in enumerate(tscv.split(X_train), 1):
    X_tr, X_val = X_train[tr_idx], X_train[val_idx]
    y_tr, y_val = y_train[tr_idx], y_train[val_idx]

    model = XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.05,
        subsample=0.9, colsample_bytree=0.9,
        scale_pos_weight=spw, objective='binary:logistic',
        eval_metric='logloss', random_state=SEED, n_jobs=-1
    )
    model.fit(X_tr, y_tr)

    y_pred = model.predict(X_val)
    prec = precision_score(y_val, y_pred, zero_division=0)
    print(f'Fold {fold}: precision = {prec:.4f}')

    if prec > best_precision:
        best_precision = prec
        best_model = model

print(f'\nBest CV precision: {best_precision:.4f}')

## Examining Feature Importances

Let's see which features the model relies on most.

In [ ]:
importances = best_model.feature_importances_
feat_imp = pd.DataFrame({'feature': input_cols, 'importance': importances})
feat_imp = feat_imp.sort_values('importance', ascending=False)
feat_imp.head(15)

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
top15 = feat_imp.head(15).sort_values('importance')
ax.barh(top15['feature'], top15['importance'], color='steelblue')
ax.set_title('Top 15 Feature Importances (XGBoost)')
ax.set_xlabel('Importance')
plt.tight_layout()
plt.show()

- ATR and volatility-related features rank highly, which aligns with our EDA findings
- The model uses a mix of momentum (RSI, MACD) and volatility features
- The macro factor (treasury yield) also contributes

# Making Predictions and Evaluating the Model

## Evaluating on Training Set

First, let's retrain the best model on the full training set and check performance.

In [ ]:
# Retrain on full training data
final_model = XGBClassifier(
    n_estimators=300, max_depth=4, learning_rate=0.05,
    subsample=0.9, colsample_bytree=0.9,
    scale_pos_weight=spw, objective='binary:logistic',
    eval_metric='logloss', random_state=SEED, n_jobs=-1
)
final_model.fit(X_train, y_train)

train_pred = final_model.predict(X_train)
print(classification_report(y_train, train_pred, zero_division=0))

In [ ]:
# Get probabilities for threshold tuning later
train_proba = final_model.predict_proba(X_train)[:, 1]

## Evaluating on Test Set

In [ ]:
X_test = test_inputs_scaled.values
y_test = test_targets.values

test_pred = final_model.predict(X_test)
print(classification_report(y_test, test_pred, zero_division=0))

In [ ]:
test_proba = final_model.predict_proba(X_test)[:, 1]

## ROC Curve

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve

In [ ]:
# ROC-AUC
try:
    auc_train = roc_auc_score(y_train, train_proba)
    auc_test = roc_auc_score(y_test, test_proba)
    print(f'ROC-AUC (train): {auc_train:.4f}')
    print(f'ROC-AUC (test):  {auc_test:.4f}')
except ValueError as e:
    print(f'Could not compute AUC: {e}')

In [ ]:
# Plot ROC curve for the test set
fpr, tpr, _ = roc_curve(y_test, test_proba)

fig, ax = plt.subplots(figsize=(8, 6))
ax.plot(fpr, tpr, label=f'XGBoost (AUC = {auc_test:.3f})', color='steelblue')
ax.plot([0, 1], [0, 1], 'k--', label='Random baseline')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve (Test Set)')
ax.legend()
plt.tight_layout()
plt.show()

## Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, test_pred)
fig, ax = plt.subplots(figsize=(6, 5))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax,
            xticklabels=['No (0)', 'Yes (1)'], yticklabels=['No (0)', 'Yes (1)'])
ax.set_xlabel('Predicted')
ax.set_ylabel('Actual')
ax.set_title('Confusion Matrix (Test Set)')
plt.tight_layout()
plt.show()

## Threshold Tuning

The default threshold of 0.5 might not be optimal. In a trading context, precision matters a lot: we'd rather miss some opportunities (lower recall) than make bad trades (low precision). Let's sweep different thresholds.

In [ ]:
from sklearn.metrics import recall_score, f1_score

thresholds = np.arange(0.30, 0.85, 0.05)
results = []

for t in thresholds:
    preds = (test_proba >= t).astype(int)
    n_signals = preds.sum()
    coverage = n_signals / len(preds)
    prec = precision_score(y_test, preds, zero_division=0)
    rec = recall_score(y_test, preds, zero_division=0)
    f1 = f1_score(y_test, preds, zero_division=0)
    results.append({'threshold': round(t, 2), 'precision': prec, 'recall': rec,
                    'f1': f1, 'signals': n_signals, 'coverage': round(coverage, 3)})

threshold_df = pd.DataFrame(results)
threshold_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(threshold_df['threshold'], threshold_df['precision'], 'o-', label='Precision', color='green')
ax.plot(threshold_df['threshold'], threshold_df['recall'], 's-', label='Recall', color='orange')
ax.plot(threshold_df['threshold'], threshold_df['coverage'], '^-', label='Coverage', color='blue')
ax.set_xlabel('Decision Threshold')
ax.set_ylabel('Score')
ax.set_title('Precision / Recall / Coverage vs Threshold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

- Precision generally improves as we raise the threshold (fewer but more confident signals)
- Recall drops because we're filtering out more predictions
- We need to find a balance $\rightarrow$ let's also look at this from a strategy perspective

# Comparing the Model's Performance to Other Models

Let's train a Logistic Regression and a Random Forest with the same data and see how they compare.

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

In [ ]:
# Logistic Regression
lr_model = LogisticRegression(solver='liblinear', class_weight='balanced', max_iter=4000, random_state=SEED)
lr_model.fit(X_train, y_train)
lr_pred = lr_model.predict(X_test)
lr_proba = lr_model.predict_proba(X_test)[:, 1]

print('=== Logistic Regression ===')
print(classification_report(y_test, lr_pred, zero_division=0))

In [ ]:
# Random Forest
rf_model = RandomForestClassifier(
    n_estimators=400, max_depth=8, min_samples_leaf=8,
    class_weight='balanced_subsample', random_state=SEED, n_jobs=-1
)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_proba = rf_model.predict_proba(X_test)[:, 1]

print('=== Random Forest ===')
print(classification_report(y_test, rf_pred, zero_division=0))

In [ ]:
# Comparison table
from sklearn.metrics import average_precision_score

models = {
    'Logistic Regression': (lr_pred, lr_proba),
    'Random Forest': (rf_pred, rf_proba),
    'XGBoost': (test_pred, test_proba)
}

comparison = []
for name, (preds, proba) in models.items():
    try:
        auc = roc_auc_score(y_test, proba)
    except:
        auc = 0
    try:
        pr_auc = average_precision_score(y_test, proba)
    except:
        pr_auc = 0

    comparison.append({
        'Model': name,
        'Precision': precision_score(y_test, preds, zero_division=0),
        'Recall': recall_score(y_test, preds, zero_division=0),
        'F1': f1_score(y_test, preds, zero_division=0),
        'ROC-AUC': auc,
        'PR-AUC': pr_auc
    })

comparison_df = pd.DataFrame(comparison).round(4)
comparison_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
comparison_df.set_index('Model')[['Precision', 'Recall', 'F1']].plot(kind='bar', ax=ax)
ax.set_title('Model Comparison on Test Set')
ax.set_ylabel('Score')
ax.legend(loc='upper right')
ax.set_xticklabels(comparison_df['Model'], rotation=0)
plt.tight_layout()
plt.show()

- We can compare precision, recall, and F1 across the three models
- For a trading strategy, precision is the most important metric because false positives cost money
- The best model depends on the trade-off we want between precision and coverage

# Backtesting the Strategy

Beyond ML metrics, we need to check whether the model actually makes money. We'll simulate a simple strategy: when the model says BUY, we go long; otherwise we stay in cash. We evaluate on non-overlapping 5-day windows to avoid inflated results.

In [ ]:
def backtest_strategy(df, proba, threshold=0.5, horizon=5):
    """Run a simple long/cash backtest on non-overlapping windows."""
    df = df.copy()
    df['proba'] = proba
    df['signal'] = (df['proba'] >= threshold).astype(int)

    # Non-overlapping windows
    df = df.iloc[::horizon].copy()
    df['strategy_return'] = df['signal'] * df['future_return']

    equity = (1 + df['strategy_return']).cumprod()

    # Annualized return
    total_ret = equity.iloc[-1] / equity.iloc[0] - 1
    n_periods = len(equity)
    periods_per_year = 252 / horizon
    years = n_periods / periods_per_year
    ann_ret = (1 + total_ret) ** (1 / max(years, 0.01)) - 1

    # Sharpe ratio
    strat_returns = df['strategy_return']
    if strat_returns.std() > 0:
        sharpe = (strat_returns.mean() / strat_returns.std()) * np.sqrt(periods_per_year)
    else:
        sharpe = 0

    # Max drawdown
    cum_max = equity.cummax()
    mdd = (equity / cum_max - 1).min()

    return {'ann_return': ann_ret, 'sharpe': sharpe, 'max_drawdown': mdd, 'equity': equity}

In [ ]:
# Backtest at different thresholds
bt_results = []
for t in [0.30, 0.35, 0.40, 0.45, 0.50, 0.55, 0.60, 0.65, 0.70]:
    res = backtest_strategy(test_df, test_proba, threshold=t)
    bt_results.append({
        'threshold': t,
        'ann_return': round(res['ann_return'], 4),
        'sharpe': round(res['sharpe'], 3),
        'max_drawdown': round(res['max_drawdown'], 4)
    })

bt_df = pd.DataFrame(bt_results)
bt_df

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(bt_df['threshold'], bt_df['ann_return'], 'o-', color='green')
ax1.set_xlabel('Threshold')
ax1.set_ylabel('Annualized Return')
ax1.set_title('Annualized Return vs Threshold')
ax1.grid(True, alpha=0.3)

ax2.plot(bt_df['threshold'], bt_df['sharpe'], 's-', color='steelblue')
ax2.set_xlabel('Threshold')
ax2.set_ylabel('Sharpe Ratio')
ax2.set_title('Sharpe Ratio vs Threshold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

- The best threshold depends on whether we prioritize returns or risk-adjusted performance (Sharpe)
- Lower thresholds generate more trades and potentially higher raw returns, but may also increase drawdown
- Higher thresholds are more selective $\rightarrow$ fewer trades but potentially better precision

## Equity Curve

Let's visualize the equity curve at the best threshold by Sharpe ratio.

In [ ]:
best_bt_row = bt_df.loc[bt_df['sharpe'].idxmax()]
best_threshold = best_bt_row['threshold']
print(f'Best threshold by Sharpe: {best_threshold}')
print(f'Sharpe: {best_bt_row["sharpe"]}, Ann. Return: {best_bt_row["ann_return"]:.2%}, Max DD: {best_bt_row["max_drawdown"]:.2%}')

In [ ]:
res = backtest_strategy(test_df, test_proba, threshold=best_threshold)
equity = res['equity'].reset_index(drop=True)

fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(equity.values, color='steelblue')
ax.axhline(y=1.0, color='gray', linestyle='--', alpha=0.5)
ax.set_title(f'Equity Curve (threshold = {best_threshold})')
ax.set_xlabel('Trading Period (5-day windows)')
ax.set_ylabel('Portfolio Value (starting at 1.0)')
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Comparing to a Dummy Strategy

Let's compare our model to two simple baselines:
- **Random guess**: randomly predict BUY or CASH with equal probability
- **Always cash**: never trade (return = 0)

In [ ]:
from sklearn.metrics import accuracy_score

def random_guess(inputs):
    return np.random.choice([0, 1], size=len(inputs))

def always_cash(inputs):
    return np.zeros(len(inputs))

np.random.seed(SEED)
rg_pred = random_guess(X_test)
ac_pred = always_cash(X_test)

best_preds = (test_proba >= best_threshold).astype(int)

print(f'Model accuracy:        {accuracy_score(y_test, best_preds):.4f}')
print(f'Random guess accuracy:  {accuracy_score(y_test, rg_pred):.4f}')
print(f'Always-cash accuracy:   {accuracy_score(y_test, ac_pred):.4f}')

The model outperforms both dummy strategies. Even the always-cash strategy can have decent accuracy when the positive class is rare, but it generates zero return.

# Feature Ablation

Let's verify whether each group of features actually contributes. We'll drop one group at a time and check how precision changes.

In [ ]:
# Define feature groups by pattern matching on column names
feature_groups = {
    'volatility_atr': [c for c in input_cols if 'atr_' in c],
    'macro_rates': [c for c in input_cols if 'ten_year_yield' in c],
    'trend_momentum': [c for c in input_cols if any(k in c for k in ['rsi_', 'macd', 'bb_', 'ma_', 'volume_z'])]
}

for group, cols in feature_groups.items():
    print(f'{group}: {len(cols)} features')

In [ ]:
ablation_results = [{'experiment': 'full_model',
                     'precision': precision_score(y_test, test_pred, zero_division=0)}]

for group, cols_to_drop in feature_groups.items():
    remaining = [c for c in input_cols if c not in cols_to_drop]
    if len(remaining) == 0:
        continue

    X_tr_abl = train_inputs_scaled[remaining].values
    X_te_abl = test_inputs_scaled[remaining].values

    abl_model = XGBClassifier(
        n_estimators=300, max_depth=4, learning_rate=0.05,
        subsample=0.9, colsample_bytree=0.9,
        scale_pos_weight=spw, objective='binary:logistic',
        eval_metric='logloss', random_state=SEED, n_jobs=-1
    )
    abl_model.fit(X_tr_abl, y_train)
    abl_pred = abl_model.predict(X_te_abl)
    prec = precision_score(y_test, abl_pred, zero_division=0)

    ablation_results.append({'experiment': f'drop_{group}', 'precision': prec})
    print(f'drop_{group}: precision = {prec:.4f}')

ablation_df = pd.DataFrame(ablation_results)
ablation_df

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
colors = ['steelblue' if x == 'full_model' else 'coral' for x in ablation_df['experiment']]
ax.barh(ablation_df['experiment'], ablation_df['precision'], color=colors)
ax.set_xlabel('Precision')
ax.set_title('Feature Ablation: Precision by Experiment')
plt.tight_layout()
plt.show()

- If dropping a feature group hurts precision, that group is valuable
- If dropping a group doesn't change (or improves) precision, it may be adding noise
- This helps us understand which features the model truly needs

# Results Summary

In [ ]:
# Final decision table
best_model_name = comparison_df.loc[comparison_df['Precision'].idxmax(), 'Model']
best_ablation = ablation_df.loc[ablation_df['precision'].idxmax(), 'experiment']

summary_table = pd.DataFrame([
    {'Decision': 'Best model (by precision)', 'Choice': best_model_name},
    {'Decision': 'Best threshold (by Sharpe)', 'Choice': str(best_threshold)},
    {'Decision': 'Best ablation experiment', 'Choice': best_ablation},
    {'Decision': 'Test Sharpe ratio', 'Choice': str(best_bt_row['sharpe'])},
    {'Decision': 'Test annualized return', 'Choice': f'{best_bt_row["ann_return"]:.2%}'},
])
summary_table

## Discussion

**Strengths:**
- Strict chronological validation prevents data leakage
- Multiple model families compared under the same conditions
- Threshold tuning with both ML metrics and strategy-level metrics (Sharpe, drawdown)
- Feature ablation verifies which feature groups genuinely help

**Weaknesses:**
- The backtest does not account for transaction costs or slippage
- Performance depends heavily on the market regime during the test period
- Probability calibration is not applied (the raw probabilities may not be well-calibrated)

**Practical takeaway:** The model works best as a decision support tool. The threshold should be chosen based on risk appetite: a lower threshold captures more opportunities but with less precision, while a higher threshold is more selective.

# Saving the Model

In [ ]:
import joblib

In [ ]:
model_artifacts = {
    'model': final_model,
    'scaler': scaler,
    'input_cols': input_cols,
    'threshold': best_threshold,
    'config': {
        'horizon_days': HORIZON,
        'success_threshold': THRESHOLD,
        'window_days': WINDOW,
        'seed': SEED
    }
}

save_path = os.path.join(data_dir, 'model_artifacts.joblib')
joblib.dump(model_artifacts, save_path)
print(f'Saved to {save_path}')

In [ ]:
# Verify we can load it back
loaded = joblib.load(save_path)
print('Loaded model type:', type(loaded['model']))
print('Number of features:', len(loaded['input_cols']))
print('Config:', loaded['config'])

## Conclusion

We built a pipeline that predicts 5-day upside moves for high-volatility stocks. The key takeaways:

- ATR-based volatility features are among the most important predictors for this regime
- Threshold tuning has a big impact on practical performance $\rightarrow$ model quality alone doesn't define trading quality
- Feature ablation showed which feature groups actually contribute vs. which ones add noise
- The strategy generates positive risk-adjusted returns on the 2023+ test period, but real-world deployment would need transaction cost modeling and continuous retraining

## Future Work

- Add transaction costs and slippage to the backtest
- Apply probability calibration (Platt scaling or isotonic regression)
- Implement walk-forward retraining to adapt to changing market conditions
- Explore alternative data sources: options flow, earnings events, intraday volatility
- Test regime-switching between high-volatility and stable-stock configurations